# MC<sup>2</sup>
This tutorial demonstrates how to use MC<sup>2</sup> (<b>M</b>Multiparty <b>C</b>ollaboration and <b>C</b>ompetition), our platform that enables collaborating parties to jointly perform analytics and train machine learning models on their sensitive data without sharing the contents of the data. 

In this tutorial, we'll break everyone into small groups -- each group will be collaborating to jointly train a decision tree model. While in practice there will exist a central enclave server controlled by no one member of the party, in this tutorial one member per group will start the RPC enclave server that enables clients to jointly orchestrate a training pipeline that will run inside an enclave. All group members will submit requests to the enclave server to jointly execute the pipeline.

MC<sup>2</sup> is open source and available on [GitHub](https://github.com/mc2-project/mc2).

# 1. Setup
We'll first need to set up your user by inputting a username, generating a keypair, generating a certificate, and generating a symmetric key.

In [12]:
%load_ext autoreload
%autoreload 2

import securexgboost as mc2
import os
from Utils import *

# TODO: Enter your username below
username = "chief"
cwd = "/home/mc2/risecamp/mc2/tutorial/"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Run this cell to generate a keypair and a certificate
generate_certificate(username)
PUB_KEY = "config/{0}.pem".format(username)
CERT_FILE = "config/{0}.crt".format(username)

Generating keypair
Generating RSA private key, 3072 bit long modulus (2 primes)
........................................................................++++
.........................................................................................................................++++
e is 3 (0x03)
Generating CSR
Signing CSR
Signature ok
subject=CN = chief
Getting CA Private Key


In [3]:
# Run this cell to generate a symmetric key
KEY_FILE = "key.txt"
mc2.generate_client_key(KEY_FILE)

## 2. Data Encryption
Use the symmetric key generated above to encrypt your data. Your training data is located at path, and your test data is located at path.

In [13]:
training_data = "data/1_2agaricus.txt.train"
# TODO: give your encrypted training data a unique name, perhaps including your username
enc_training_data = cwd + "data/{}_train.enc".format(username)

# Encrypt training data
mc2.encrypt_file(training_data, enc_training_data, KEY_FILE)

In [14]:
test_data = "data/agaricus.txt.test"
enc_test_data = cwd + "data/{}_test.enc".format(username)

# Encrypt test data
mc2.encrypt_file(test_data, enc_test_data, KEY_FILE)

In [15]:
# Store variables for use in subsequent notebooks
%store username
%store PUB_KEY 
%store CERT_FILE 
%store KEY_FILE 
%store enc_training_data 
%store enc_test_data

Stored 'username' (str)
Stored 'PUB_KEY' (str)
Stored 'CERT_FILE' (str)
Stored 'KEY_FILE' (str)
Stored 'enc_training_data' (str)
Stored 'enc_test_data' (str)


## 3. Enclave server setup
While in practice there'll be an enclave server controlled by no one party, to complete this tutorial one party in the collaboration will have to act as both a party and the enclave server. Designate one person in the collaboration to control the enclave server.

If you've been designated as the enclave server, click [here](./rpc-orchestrator.ipynb) to go to the next notebook. You'll have to set up the enclave server before everyone can begin training. 

Otherwise, click [here](./tutorial-client.ipynb).